In [ ]:
import arkouda as ak
import arachne as ar
import time as time


In [ ]:
ak.connect("n116", 5555)
ak.client.maxTransferBytes *= 2

In [3]:
# connectome_edges = ak.read("/scratch/users/oaa9/experimentation/data/connectome/h01/edges*")
# connectome_edges = ak.DataFrame(connectome_edges)

In [4]:
# vals = connectome_edges["confidence"] > 0.75
# ak.value_counts(vals)

In [5]:
# connectome_edges["bool_conf"] = vals

In [6]:
# connectome_nodes = ak.read("/scratch/users/oaa9/experimentation/data/connectome/h01/nodes*")
# connectome_nodes = ak.DataFrame(connectome_nodes)

In [7]:
# nodes_gb = connectome_nodes.groupby("neuron_id")
# connectome_nodes = connectome_nodes[nodes_gb.permutation[nodes_gb.segments]]

In [8]:
# graph = ar.PropGraph()
# graph.load_edge_attributes(connectome_edges, source_column="src_neuron_id", destination_column="dst_neuron_id", relationship_columns="type_label")
# graph.load_node_attributes(connectome_nodes, node_column="neuron_id")

In [9]:
# print(f"Graph has {len(graph):_} vertices and {graph.size():_} edges.")

In [10]:
# src = ak.array([0, 1, 2, 2])
# dst = ak.array([1, 2, 0, 3])
# nodes = ak.array([0, 1, 2, 3])

In [11]:
# bool_conf = ak.array([True, True, True, True])

In [12]:
# subgraph_edge_df = ak.DataFrame({"src":src, "dst":dst, "bool_conf":bool_conf})

In [13]:
# subgraph = ar.PropGraph()
# subgraph.load_edge_attributes(subgraph_edge_df, source_column="src", destination_column="dst")

In [14]:
def vf2_si(g, h):
    isos_as_vertices = ar.subgraph_isomorphism(g, h, 
                                            semantic_check = "and", algorithm_type = "si",
                                            reorder_type = "structural", return_isos_as = "vertices")
    print(f"We found {len(isos_as_vertices[0])/len(h)} monos inside of the graph")

In [15]:
# def vf2_si_probability_reordering(g,h):
#     isos_as_vertices = ar.subgraph_isomorphism(g, h, 
#                                             semantic_check = "and", algorithm_type = "si",
#                                             reorder_type = "probability", return_isos_as = "vertices")
#     print(f"We found {len(isos_as_vertices[0])/len(h)} monos inside of the graph")

In [16]:
# vf2_si_probability_reordering(graph, subgraph)

In [17]:
connectome_edges = ak.read("/scratch/users/oaa9/experimentation/data/connectome/h01/edges*")
connectome_edges = ak.DataFrame(connectome_edges)

In [18]:
graph = ar.PropGraph()
graph.load_edge_attributes(connectome_edges, source_column="src_neuron_id", destination_column="dst_neuron_id")

In [19]:
num_nodes = len(graph)
num_edges = graph.size()

In [20]:
P_Alpha = 0.55
P_Beta = 0.45
node_lbl_probs = {
    "lbls2": [P_Alpha, P_Beta],  # Probabilities for integers 10 and 11
    "lbls3": [P_Alpha, P_Beta]   # Probabilities for True and False
}
edge_rel_probs = {
    "rels1": [P_Alpha, P_Beta],  # Probabilities for integers 5 and 6
    "rels2": [P_Alpha, P_Beta]   # Probabilities for True and False
}

In [21]:
# Randomly generate node attributes.
node_lbls2 = ak.where(ak.randint(0, 100, num_nodes) < node_lbl_probs["lbls2"][0] * 100, 10, 11)
node_lbls3 = ak.randint(0, 100, num_nodes) < node_lbl_probs["lbls3"][0] * 100

# Randomly generate edge attributes
edge_rels1 = ak.where(ak.randint(0, 100, num_edges) < edge_rel_probs["rels1"][0] * 100, 5, 6)
edge_rels2 = ak.randint(0, 100, num_edges) < edge_rel_probs["rels2"][0] * 100

In [22]:
# Create edge dataframe.
edge_df = ak.DataFrame({
    "src": graph.edge_attributes["src_neuron_id"],
    "dst": graph.edge_attributes["dst_neuron_id"],
    "rels1": edge_rels1,
    "rels2": edge_rels2
})

# Create node dataframe.
node_df = ak.DataFrame({
    "nodes": graph.nodes(),
    "lbls2": node_lbls2,
    "lbls3": node_lbls3
})

In [23]:
graph = ar.PropGraph()
graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
graph.load_node_attributes(node_df, node_column="nodes")

In [24]:
def save_graph_to_csv(graph, file_name):
    """
    Save a graph in CSV format with directed edges and optional labels.
    :param graph: Arachne property graph.
    :param file_name: Output CSV file name.
    """
    # Extract internal edges and attributes
    internal_src, internal_dst = graph._internal_edges()
    src = internal_src.to_list()
    dst = internal_dst.to_list()

    # Extract edge attributes
    graph_edge_attributes = graph.get_edge_attributes()
    edge_df = graph_edge_attributes.to_pandas()
    edge_rels1 = edge_df['rels1'] if 'rels1' in edge_df.columns else None
    edge_rels2 = edge_df['rels2'] if 'rels2' in edge_df.columns else None
    
    with open(file_name, "w") as f:
        print("Writing edge data")
        for i in range(len(src)):
            if edge_rels1 is not None and edge_rels2 is not None:
                f.write(f"{src[i]}>{dst[i]},{edge_rels1[i]},{edge_rels2[i]}\n")
            else:
                f.write(f"{src[i]}>{dst[i]}\n")

        print("Writing node data...")
        graph_node_attributes = graph.get_node_attributes()
        if graph_node_attributes.size > 0:
            node_df = graph_node_attributes.to_pandas()
            for index, row in node_df.iterrows():
                f.write(f"{index},,{row['lbls2']},{row['lbls3']}\n")
        else:
            # Use default labels if no node attributes are available
            unique_nodes = sorted(set(src).union(dst))
            for node in unique_nodes:
                f.write(f"{node},,\n")

    print(f"Graph saved to {file_name}")

In [25]:
src0 = [    1,     1]
dst0 = [10002, 10003]
connection_type0 = [0, 0]

src1 = [    1,     1, 10003, 10003]
dst1 = [10002, 10003, 30004, 30005]
connection_type1 = [0, 0, 0, 0]

src2 = [    1,     1, 40005]
dst2 = [10002, 10003,     1]
connection_type2 = [0, 0, 1]

src3 = [    1,     1, 40005, 40005, 50008]
dst3 = [10002, 10003, 50008,     1, 10003]
connection_type3 = [0, 0, 0, 1, 1]

src4 = [1, 1, 10003, 10003, 60007]
dst4 = [10002, 10003, 30004, 30005, 1]
connection_type4 = [0, 0, 0, 0, 1]

src5 = [1, 1, 10003, 10003, 60007, 60007, 70010]
dst5 = [10002, 10003, 30004, 30005, 70010, 1, 30005]
connection_type5 = [0, 0, 0, 0, 0, 1, 1]

src6 = [1, 1, 40005, 40005, 80009, 80009, 10003, 90010]
dst6 = [10002, 10003, 50006, 50007, 90010, 90011, 50006, 50007]
connection_type6 = [0, 0, 0, 0, 0, 0, 1, 1]

src7 = [1, 10002, 40005, 60007, 80009]
dst7 = [10002, 20003, 1, 10002, 20003]
connection_type7 = [0, 0, 1, 1, 1]

src00 = [1, 1, 1, 1, 10002]
dst00 = [10002, 10003, 10004, 50006, 70008]
connection_type00 = [0, 0, 0, 1, 1]

src01 = [1, 10002, 20003, 30004, 1, 40005]
dst01 = [10002, 20003, 30004, 40005, 60007, 80009]
connection_type01 = [0, 0, 0, 0, 1, 1]

src02 = [1, 30004, 60007, 90010, 110012, 130014]
dst02 = [10002, 40005, 70008, 1, 30004, 60007]
connection_type02 = [0, 0, 0, 1, 1, 1]

src03 = [1, 1, 1, 50006, 50006, 50006, 120013, 120013, 120013, 10003, 10004]
dst03 = [10002, 10003, 10004, 60007, 60008, 60009, 130014, 130015, 130016, 60007, 130015]

In [ ]:
# src_list = [1, 0, 1]
# dst_list = [0, 1, 2]
src_list = src7
dst_list = dst7
src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)

subgraph_nodes = list(set(src_list) | set(dst_list))
subgraph_nodes.sort()

# Generate random node and edge attributes for the subgraph
num_subgraph_nodes = len(subgraph_nodes)
num_subgraph_edges = len(src_list)

subgraph_node_ints = ak.array([11]*num_subgraph_nodes)
subgraph_node_bools = ak.array([False]*num_subgraph_nodes)
subgraph_edge_ints = ak.array([6]*num_subgraph_edges)
subgraph_edge_bools = ak.array([False]*num_subgraph_edges)

# Create dataframes for subgraph attributes
edge_df_h = ak.DataFrame({
    "src": src_subgraph,
    "dst": dst_subgraph,
    "rels1": subgraph_edge_ints,
    "rels2": subgraph_edge_bools
})

node_df_h = ak.DataFrame({
    "nodes": ak.array(subgraph_nodes),
    "lbls2": subgraph_node_ints,
    "lbls3": subgraph_node_bools
})

# Create the subgraph with these attributes
subgraph = ar.PropGraph()
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
subgraph.load_node_attributes(node_df_h, node_column="nodes")



print("Subgraph created with fixed node and edge attributes.")
print(" Run Arachne....")

In [ ]:
"""VF2-SI """
start_time = time.time()

isos_as_vertices = ar.subgraph_isomorphism(graph, subgraph, 
                                           semantic_check = "and", algorithm_type = "si",
                                           reorder_type = "structural", return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(subgraph)} monos inside of the graph")
# Stop the timer
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print results
num_monos = len(isos_as_vertices[0]) / len(subgraph) if len(subgraph) > 0 else 0
print(f"VF2-SI completed in {elapsed_time:.4f} seconds.")
print(f"We found {num_monos} monos inside of the graph.")

In [28]:
# # save_graph_to_csv(graph, "H01.csv")
# # # Save the main graph to a CSV file
# print("Processing main graph...")
# save_graph_to_csv(graph, "main_graph_H01-1.csv")
# print("Main graph saved ")



In [29]:
# # Save the subgraph to a CSV file
# print("Processing subgraph...")
# save_graph_to_csv(subgraph, "H01-1.csv")
# print("subgraph saved  ....")

In [30]:


# """VF2-PS DEFAULT"""
# print("Running VF2-PS DEFAULT...")
# start_time = time.time()

# isos_as_vertices = ar.subgraph_isomorphism(
#     graph, subgraph,
#     semantic_check="and", algorithm_type="ps",
#     reorder_type=None, return_isos_as="vertices"
# )

# end_time = time.time()
# elapsed_time = end_time - start_time

# num_monos = len(isos_as_vertices[0]) / len(subgraph) if len(subgraph) > 0 else 0
# print(f"VF2-PS DEFAULT completed in {elapsed_time:.4f} seconds.")
# print(f"We found {num_monos} monos inside of the graph.")
# # ak.shutdown()

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt

# # Define the source and destination lists
# src03 = [1, 1, 1, 50006, 50006, 50006, 120013, 120013, 120013, 10003, 10004]
# dst03 = [10002, 10003, 10004, 60007, 60008, 60009, 130014, 130015, 130016, 60007, 130015]

# # Create a directed graph
# graph = nx.DiGraph()

# # Add edges to the graph
# edges = zip(src03, dst03)
# graph.add_edges_from(edges)

# # Draw the graph
# plt.figure(figsize=(10, 6))
# pos = nx.spring_layout(graph)  # Layout for visualization
# nx.draw(graph, pos, with_labels=True, node_color="lightblue", edge_color="gray", node_size=800, font_size=10)
# nx.draw_networkx_edge_labels(graph, pos, edge_labels={(u, v): f"{u}->{v}" for u, v in graph.edges()}, font_size=8)

# # Show the graph
# plt.title("Graph Visualization")
# plt.show()
